You need to have IPTA DR2 data somewhere on your machine.  You can pull it from `gitlab`:
```bash
$ git clone https://gitlab.com/IPTA/DR2
```

In [ ]:
from __future__ import division, print_function, unicode_literals

import numpy as np
import os, glob
import json
import matplotlib.pyplot as plt
import libstempo as t2

import dr2lite_utils as dr2u

%matplotlib inline

# 1) Clean par and tim files

1. Make a directory called `partim` that contains a single par file and single tim file for all pulsars.

2. Clean the par files by removing all noise and DM model parameters. We will also add in DM1 and DM2 if not already present in the par file.

3. Concatenate all "INCLUDEs" into a single tim file

This uses `clear_par()` and `combine_tim()` from `dr2lite_utils`

### Create partim directory

In [ ]:
DR2DATA = os.path.abspath('/home/nanograv/local_data/IPTA_DR2/')  # path to data for Docker image
datadir = os.path.join(DR2DATA, 'release/VersionA')

outdir = 'partim'
os.system('mkdir -p {}'.format(outdir));

### Clean par files

In [ ]:
parfiles = glob.glob(datadir + '/J*/*IPTADR2.par')

psr_names = []
for p in parfiles:
    name = p.split('/')[-2]
    psr_names.append(name)
    outfile = os.path.join(outdir, '{}.par'.format(name))
    dr2u.clean_par(p, outfile)

### Combine tim files

In [ ]:
timfiles = glob.glob(datadir + '/J*/*IPTADR2.tim')

for t in timfiles:
    name = t.split('/')[-2]
    outfile = os.path.join(outdir, '{}.tim'.format(name))
    dr2u.combine_tim(t, outfile)

# 2.1) How to create filtered par and tim files

Here we create a filtered dataset by applying a frequency filter and a PTA filter

1. **Frequency filter**: Only keep TOAs that have a certain bandwidth (`bw`) coverage over a certain period of time (`dt`). For instance, `dt=7` and `bw=800` will only keep TOAs that have at least 800 MHz of bandwidth (i.e. difference between max and min radio frequency) in every 7 day window.

2. **Flag filter**: Only keep TOAs with a give flag value.  For example keep only TOAs with flag `pta` for a subset of PTAs or flag `group` for a subset of backends.

These steps use the functions provided in `dr2lite_utils.py`

### Filter dictionary
This is input to `filter_psr()`.  This is a dictionary where the keys are TOA flags and the values are lists of acceptable flagvals to keep.  If a single flagval is to be kept, it my be passed as is (not as a list).

For a given flag (e.g. `pta`), filters are applied as a `logical_or`.  For a TOA to be kept, it must pass all individual filters as a `logical_and`.

**examples:**

 * ```python
fliter_dict = {'pta':'NANOGrav', 'group':['Rcvr_800_GUPPI', 'Rcvr1_2_GUPPI']}
```
This would keep only toas from NANOGrav with 800 MHz GUPPI **OR** L-band GUPPI.  Since only NANOGrav uses GUPPI, the `pta` flag is redundant.  Since PTA filters are the most common, we choose to always use a PTA filter for clarity.

 * ```python
fliter_dict = {'pta':['PPTA', 'EPTA']}
```
This would keep all PPTA and EPTA data for the pulsar.

 * ```python
fliter_dict = {'pta':'PPTA', 'group':'Rcvr_1_2_GUPPI}
```
This would keep any PPTA data that uses the L-band GUPPI backend... no TOAs would be returned.

### Pulsar dictionary

This is input to the `make_dataset()` function.  `make_dataset()` loops over a list of pulsars and runs `filter_psr()` on each.

Ths pulsar dictionary is a dictionary of filters.  The keys are the pulsars we wish to use, and the values are the `filter_dict`s to use for that pulsar.

### Example pulsar dictionaries

We define several pulsar dictionaries that can be used to generate filtered DR2-lite datasets.

In [ ]:
# NG9 + 0437
# to exclude pre-XASP NG data, we must list all of the XASP/XUPPI backends to keep
psrdict = {'J1713+0747': {'pta': ['NANOGrav'], 
                          'f': ['Rcvr1_2_GASP', 'Rcvr1_2_GUPPI', 
                                 'Rcvr_800_GASP', 'Rcvr_800_GUPPI', 
                                 'L-wide_ASP', 'L-wide_PUPPI','S-wide_ASP', 
                                 'S-wide_PUPPI']},
           'J1909-3744': {'pta': ['NANOGrav']},
           'J1640+2224': {'pta': ['NANOGrav']},
           'J1600-3053': {'pta': ['NANOGrav']},
           'J2317+1439': {'pta': ['NANOGrav']},
           'J1918-0642': {'pta': ['NANOGrav']},
           'J1614-2230': {'pta': ['NANOGrav']},
           'J1744-1134': {'pta': ['NANOGrav']},
           'J0030+0451': {'pta': ['NANOGrav']},
           'J2145-0750': {'pta': ['NANOGrav']},
           'J1857+0943': {'pta': ['NANOGrav']},
           'J1853+1303': {'pta': ['NANOGrav']},
           'J0613-0200': {'pta': ['NANOGrav']},
           'J1455-3330': {'pta': ['NANOGrav']},
           'J1741+1351': {'pta': ['NANOGrav']},
           'J2010-1323': {'pta': ['NANOGrav']},
           'J1024-0719': {'pta': ['NANOGrav']},
           'J1012+5307': {'pta': ['NANOGrav']},
           'J0437-4715': {'pta': ['PPTA']},
          }

In [ ]:
# NG9 + PPTA4
psrdict = {'J1713+0747': {'pta': ['NANOGrav', 'PPTA'], 
                          'f': ['Rcvr1_2_GASP', 'Rcvr1_2_GUPPI', 
                                'Rcvr_800_GASP', 'Rcvr_800_GUPPI', 
                                'L-wide_ASP', 'L-wide_PUPPI','S-wide_ASP', 
                                'S-wide_PUPPI', '1050CM_CPSR2', '1050CM_PDFB1', 
                                '1050CM_PDFB2', '1050CM_PDFB3',
                                '1050CM_PDFB4', '1050CM_WBCORR', 'H-OH_CPSR2m', 'H-OH_CPSR2n',
                                'H-OH_PDFB1', 'MULTI_CPSR2m', 'MULTI_CPSR2n', 'MULTI_PDFB1',
                                'MULTI_PDFB2', 'MULTI_PDFB3', 'MULTI_PDFB4', 'MULTI_WBCORR']},
           'J1909-3744': {'pta': ['NANOGrav', 'PPTA']},
           'J1640+2224': {'pta': ['NANOGrav']},
           'J1600-3053': {'pta': ['NANOGrav']},
           'J2317+1439': {'pta': ['NANOGrav']},
           'J1918-0642': {'pta': ['NANOGrav']},
           'J1614-2230': {'pta': ['NANOGrav']},
           'J1744-1134': {'pta': ['NANOGrav', 'PPTA']},
           'J0030+0451': {'pta': ['NANOGrav']},
           'J2145-0750': {'pta': ['NANOGrav']},
           'J1857+0943': {'pta': ['NANOGrav']},
           'J1853+1303': {'pta': ['NANOGrav']},
           'J0613-0200': {'pta': ['NANOGrav']},
           'J1455-3330': {'pta': ['NANOGrav']},
           'J1741+1351': {'pta': ['NANOGrav']},
           'J2010-1323': {'pta': ['NANOGrav']},
           'J1024-0719': {'pta': ['NANOGrav']},
           'J1012+5307': {'pta': ['NANOGrav']},
           'J0437-4715': {'pta': ['PPTA']},
          }

In [ ]:
# NG9 with 1713, 1744, 1909, 0437 from PPTA
psrdict = {'J1713+0747': {'pta': ['PPTA']},
           'J1909-3744': {'pta': ['PPTA']},
           'J1640+2224': {'pta': ['NANOGrav']},
           'J1600-3053': {'pta': ['NANOGrav']},
           'J2317+1439': {'pta': ['NANOGrav']},
           'J1918-0642': {'pta': ['NANOGrav']},
           'J1614-2230': {'pta': ['NANOGrav']},
           'J1744-1134': {'pta': ['PPTA']},
           'J0030+0451': {'pta': ['NANOGrav']},
           'J2145-0750': {'pta': ['NANOGrav']},
           'J1857+0943': {'pta': ['NANOGrav']},
           'J1853+1303': {'pta': ['NANOGrav']},
           'J0613-0200': {'pta': ['NANOGrav']},
           'J1455-3330': {'pta': ['NANOGrav']},
           'J1741+1351': {'pta': ['NANOGrav']},
           'J2010-1323': {'pta': ['NANOGrav']},
           'J1024-0719': {'pta': ['NANOGrav']},
           'J1012+5307': {'pta': ['NANOGrav']},
           'J0437-4715': {'pta': ['PPTA']},
          }

In [ ]:
# PPTA 10cm data for Shannon, et al (2015)
psrdict = {'J1713+0747': {'pta': ['PPTA']}, {'group': ['PDFB_10CM', 'WBCORR_10CM']},
           'J1909-3744': {'pta': ['PPTA']}, {'group': ['PDFB_10CM', 'WBCORR_10CM']},
           'J1744-1134': {'pta': ['PPTA']}, {'group': ['PDFB_10CM', 'WBCORR_10CM']},
           'J0437-4715': {'pta': ['PPTA']}, {'group': ['PDFB_10CM', 'WBCORR_10CM']},
          }

# 2.2) Generate DR2-lite style dataset

In [ ]:
# This generates an all EPTA `psrdict`
# or make your own, or choose from above
psrdict = {}  
PSRnames = glob.glob('partim/*.par')

for psrname in PSRnames:
    name = psrname.split('/')[-1].split('.')[0]
    psrdict.update({name: [{'pta':['EPTA']}]})

In [ ]:
dr2u.make_dataset(psrdict, outdir='partim_epta')